In [1]:
from __future__ import annotations

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

from helper import (
    start_time,
    time_since,
    ShakespeareDataset,
    TokenMapping,
    build_model,
    next_token,
    # Character-based helpers
    encode_text,
    # Subword-based helpers
    encode_text_from_tokenizer,
    tokenize_text_from_tokenizer,
)

c:\Users\freda\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Deterministic training
torch.manual_seed(0)

# Attempt GPU; if not, stay on CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# Load Data

In [5]:
# Reduced data to make it manageable for smaller systems
DATA_FILE: str = '../data/shakespeare_small.txt'

with open(DATA_FILE, 'r') as data_file:
    raw_text = data_file.read()

print(f'Number of characters in text file: {len(raw_text):,}')

Number of characters in text file: 9,771


# Character-Based Text Generation

The first model you'll build for text generation will use character-based
tokens.

Each token will be a single character from the text and the model will learn
to predict the next character (a token).

To generate text, the model will take in a new string,
character-by-character, and then generate a new likely character based on the
past input. Then the model will take into account that new character and
generate the following character and so on and so on until the model has
produced a set number of characters.

## Encode Text into Integer Tokens

### Normalization

In [6]:
import string
def normalize_text(text: str) -> str:
    # TODO: Normalize incoming text; can be multiple actions
    return text.lower()

In [7]:
# TEST: Is your text normalized the way you expected?
# Only the first 500 characters of the original text
normalized_text = normalize_text(raw_text[:500])
print(normalized_text)

prince. where are the vile beginners of this fray?
  ben. o noble prince. i can discover all
    the unlucky manage of this fatal brawl.
    there lies the man, slain by young romeo,
    that slew thy kinsman, brave mercutio.
  cap. wife. tybalt, my cousin! o my brother's child!
    o prince! o husband! o, the blood is spill'd
    of my dear kinsman! prince, as thou art true,
    for blood of ours shed blood of montague.
    o cousin, cousin!
  prince. benvolio, who began this bloody fray?
  ben


### Pretokenization

In [8]:
def pretokenize_text(text: str) -> str | list[str]:
    # TODO: Pretokenize normalized text into character strings
    smaller_pieces = [char for char in text]
    return smaller_pieces

In [9]:
# TEST: Is your (normalized) text pretokenized the way you expected?
# Only the first 500 characters of the original text
pretokenized_text = pretokenize_text(normalized_text)
print(pretokenized_text)

['p', 'r', 'i', 'n', 'c', 'e', '.', ' ', 'w', 'h', 'e', 'r', 'e', ' ', 'a', 'r', 'e', ' ', 't', 'h', 'e', ' ', 'v', 'i', 'l', 'e', ' ', 'b', 'e', 'g', 'i', 'n', 'n', 'e', 'r', 's', ' ', 'o', 'f', ' ', 't', 'h', 'i', 's', ' ', 'f', 'r', 'a', 'y', '?', '\n', ' ', ' ', 'b', 'e', 'n', '.', ' ', 'o', ' ', 'n', 'o', 'b', 'l', 'e', ' ', 'p', 'r', 'i', 'n', 'c', 'e', '.', ' ', 'i', ' ', 'c', 'a', 'n', ' ', 'd', 'i', 's', 'c', 'o', 'v', 'e', 'r', ' ', 'a', 'l', 'l', '\n', ' ', ' ', ' ', ' ', 't', 'h', 'e', ' ', 'u', 'n', 'l', 'u', 'c', 'k', 'y', ' ', 'm', 'a', 'n', 'a', 'g', 'e', ' ', 'o', 'f', ' ', 't', 'h', 'i', 's', ' ', 'f', 'a', 't', 'a', 'l', ' ', 'b', 'r', 'a', 'w', 'l', '.', '\n', ' ', ' ', ' ', ' ', 't', 'h', 'e', 'r', 'e', ' ', 'l', 'i', 'e', 's', ' ', 't', 'h', 'e', ' ', 'm', 'a', 'n', ',', ' ', 's', 'l', 'a', 'i', 'n', ' ', 'b', 'y', ' ', 'y', 'o', 'u', 'n', 'g', ' ', 'r', 'o', 'm', 'e', 'o', ',', '\n', ' ', ' ', ' ', ' ', 't', 'h', 'a', 't', ' ', 's', 'l', 'e', 'w', ' ', 't', 'h', 

### Tokenize

In [10]:
# Combine normalization and pretokenization steps
def tokenize_text(text: str) -> str | list[str]:
    normalized_text: str = normalize_text(text)
    pretokenized_text: str | list[str] = pretokenize_text(normalized_text)
    # Characters are already tokens so pretokenized text is already tokenized
    tokenized_text = pretokenized_text
    return tokenized_text

In [11]:
# TEST: Is your tokenized text the way you expected?
tokenized_text = tokenize_text(raw_text[:500])
print(tokenized_text)

['p', 'r', 'i', 'n', 'c', 'e', '.', ' ', 'w', 'h', 'e', 'r', 'e', ' ', 'a', 'r', 'e', ' ', 't', 'h', 'e', ' ', 'v', 'i', 'l', 'e', ' ', 'b', 'e', 'g', 'i', 'n', 'n', 'e', 'r', 's', ' ', 'o', 'f', ' ', 't', 'h', 'i', 's', ' ', 'f', 'r', 'a', 'y', '?', '\n', ' ', ' ', 'b', 'e', 'n', '.', ' ', 'o', ' ', 'n', 'o', 'b', 'l', 'e', ' ', 'p', 'r', 'i', 'n', 'c', 'e', '.', ' ', 'i', ' ', 'c', 'a', 'n', ' ', 'd', 'i', 's', 'c', 'o', 'v', 'e', 'r', ' ', 'a', 'l', 'l', '\n', ' ', ' ', ' ', ' ', 't', 'h', 'e', ' ', 'u', 'n', 'l', 'u', 'c', 'k', 'y', ' ', 'm', 'a', 'n', 'a', 'g', 'e', ' ', 'o', 'f', ' ', 't', 'h', 'i', 's', ' ', 'f', 'a', 't', 'a', 'l', ' ', 'b', 'r', 'a', 'w', 'l', '.', '\n', ' ', ' ', ' ', ' ', 't', 'h', 'e', 'r', 'e', ' ', 'l', 'i', 'e', 's', ' ', 't', 'h', 'e', ' ', 'm', 'a', 'n', ',', ' ', 's', 'l', 'a', 'i', 'n', ' ', 'b', 'y', ' ', 'y', 'o', 'u', 'n', 'g', ' ', 'r', 'o', 'm', 'e', 'o', ',', '\n', ' ', ' ', ' ', ' ', 't', 'h', 'a', 't', ' ', 's', 'l', 'e', 'w', ' ', 't', 'h', 

### Postprocessing

We'll skip postprocessing since we don't have any special tokens we want to
consider for our task here.

### Encode (Tokens → Integer IDs)

We have `encode_text()` from our helper module that can encode our text based on
our tokenization process from our created `tokenize_text()` function.

This will also provide us with `character_mapping`, an object that we can use to
map our tokens back and forth from strings to integer IDs.

In [12]:
encoded_text, character_mapping = encode_text(raw_text, tokenize_text)

## Prepare Dataset

In [13]:
n_tokens = character_mapping.n_tokens
dataset_size = len(encoded_text)
print(f'Size of dataset: {dataset_size:,} characters')

Size of dataset: 9,772 characters


In [14]:
# Defining sequence length that will be taken in at a time by our model
sequence_length = 32 # Number of characters
batch_size = 32

train_dataset = ShakespeareDataset(encoded_text, sequence_length)
train_loader = DataLoader(
    train_dataset,
    shuffle=False, # Ensure deterministic training
    batch_size=batch_size,
)

## Define Model

We'll provide a defined model today, but this could be a step that you would
modify and experiment in other NLP projects you'll do.

In [15]:
# Defining the model to be trained and generate text with
model = build_model(n_tokens)
model.to(device)
print(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

cuda:0


## Define Text Generation

The `generate_text_by_char()` function will use your tokenizer and NLP model to
generate new text token-by-token (character-by-character in this case) by taking
in the input text and token sampling parameters.

We can use temperature and top-k sampling to adjust the "creativeness" of the
generated text.

We also pass in the `num_chars` parameter to tell the function how many tokens
(characters in this case) to generate.

In [17]:
def generate_text_by_char(
    input_str: str,
    model,
    token_mapping: TokenMapping = character_mapping,
    num_chars: int = 100,
    temperature: float = 1.0,
    topk: int | None = None,
) -> str:
    # Uses your character-based tokenizer
    tokenized_text: list[str] = tokenize_text(input_str)
    # Generates token-by-token and creates a list of those tokens
    generated_tokens = []
    for _ in range(num_chars):
        # Uses the input text and generated text (so far) to get next token
        new_char = next_token(
            tokenized_text=(tokenized_text + generated_tokens),
            model=model,
            token_mapping=token_mapping,
            # Temperature & top-k sampling used in determining the next token
            temperature=temperature,
            topk=topk,
            device=device,
        )
        generated_tokens.append(new_char)
    # Returns input string plus the full generated string (of generated tokens)
    full_text = ''.join(tokenized_text + generated_tokens)
    return full_text

## Train Model

At this point, the model has not been trained so the code below will train the
NLP model that will be used to generate new text.

The model will take in the text data (broken by tokens by our character-based
tokenizer) and attempt to predict the next token. Over time, the model should
hopefully get better in predicting the next token (given the previous text).

To help us visualize how the model is training, at the end of every epoch, we
generate text using the `TEST_PHRASE` with the improving model.

In [18]:
TEST_PHRASE = 'To be or not to be'
# Use more epochs if not CPU device
epochs = 5 if device == 'cpu' else 25

start = start_time()
for epoch in range(epochs):
    # Set model into "training mode"
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.to(device))
        loss = criterion(output.transpose(1, 2), y_batch.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')
    print(f'[{time_since(start)} ({epoch} {epoch / epochs * 100}) {loss:.4f}]')
    print('-'*72)
    gen_output = generate_text_by_char(
        input_str=TEST_PHRASE,
        model=model,
        num_chars=100,
    )
    print(gen_output)

Epoch 1/25, Loss: 2.9390148670946967
 1.6s (0 0.0) 2.7030]
------------------------------------------------------------------------
to be or not to bed y  oc.o
   i yorethi'oai iaha,  womwhthroa

f  bie
 i
t  ton whe. y mard
-nec

  thetgiatorm l.end
Epoch 2/25, Loss: 2.5208502706934195
 2.9s (1 4.0) 2.4422]
------------------------------------------------------------------------
to be or not to bes f    has measutherds,  ot orrase sillj
 erh a, beo slth!
    ka
   jtow ovse an d thant thael thed
Epoch 3/25, Loss: 2.3508560430808148
 4.1s (2 8.0) 2.3152]
------------------------------------------------------------------------
to be or not to bemd, h eek,
      meut, pdeo sujhe.and
 y ind ootryhither m? spxthoth fis -for pasthe ee agj, nom yis
Epoch 4/25, Loss: 2.2578898824629237
 5.3s (3 12.0) 2.2559]
------------------------------------------------------------------------
to be or not to be,  hand, what hor it wehat depod rod! youw
    sil  me  fall luf pe ronln.
    sghou of tbwor wo

## Generate Text

Now that the model has been trained, go ahead and observe how it performs!

Try adjusting the different sampling methods using the `temperature` and `topk`
parameters on the same input string to see the differences.

You might also try different phrases as well as how many tokens (`num_chars`) to
generate and observe how it does.

In [19]:
output = generate_text_by_char(
    input_str='To be or not to be',
    model=model,
    num_chars=100,
    temperature=1.0,
    topk=None,
)
print(output)

to be or not to bere. wake o hnant hath mayl fot mof! nemiod, wit, maye bet weak thes beath teach with loven
    blood


# Subword Text Generation

The next model you'll build will use subword-tokenization instead of 
characters-based token to train a model and ultimately generate new text
token-by-token.

Although this could be done by creating your own tokenizer, you'll use
Hugging Face to use a pretrained tokenizer to tokenize the data.

After training the model with subword tokens, 
the model will take in a new string, token-by-token, and then generate a new
token (subword).
The model will continue producing new subword tokens based on the input text
and already produced tokens until a set number of tokens have been generated.

## Encode Text into Integer Tokens

### Choosing a Tokenizer

> NOTE:
> 
> You can load another model outside of these choices but the model
> will have to be downloaded and may or may not be effective.
>
> If you'd like to explore more, here's a link to you might want to start with
> of different available pretrained models on Hugging Face:
> https://huggingface.co/models?pipeline_tag=text-generation

In [20]:
# TODO: Choose a pretrained tokenizer to use:

# Docs: https://huggingface.co/xlm-roberta-base
# model_name = 'xlm-roberta-base'
# DOCS: https://huggingface.co/bert-base-cased
# model_name = 'bert-base-cased'
# DOCS: https://huggingface.co/bert-base-uncased 
# model_name = 'bert-base-uncased'
model_name = 'bert-base-uncased'
my_tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)

### Encode (Tokens → Integer IDs)

We have `encode_text_from_tokenizer()` from our helper module that can encode
our text based on our tokenization process from our tokenizer `my_tokenizer`.

This will also provide us with `token_mapping`, an object that we can use to
map our tokens back and forth from strings to integer IDs.

In [21]:
encoded_text, token_mapping = encode_text_from_tokenizer(
    text=raw_text,
    tokenizer=my_tokenizer,
)

## Prepare Dataset

In [22]:
n_tokens = token_mapping.n_tokens
dataset_size = len(encoded_text)
print(f'Size of dataset: {dataset_size:,} tokens')

Size of dataset: 2,456 tokens


In [23]:
# Defining sequence length that will be taken in at a time by our model
sequence_length = 32 # Number of tokens
batch_size = 32

train_dataset = ShakespeareDataset(encoded_text, sequence_length)
train_loader = DataLoader(
    train_dataset,
    shuffle=False, # Ensure deterministic training
    batch_size=batch_size,
)

## Define Model

We'll provide a defined model today, but this could be a step that you would
modify and experiment in other NLP projects you'll do.

In [24]:
# Defining the model to be trained and generate text with
model = build_model(n_tokens)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

## Define Text Generation

The `generate_text_by_subword()` function will use your chosen tokenizer and the
NLP model to generate new text token-by-token (subwords in this case) by taking
in the input text and token sampling parameters.

We can use temperature and top-k sampling to adjust the "creativeness" of the
generated text.

We also pass in the `num_tokens` parameter to tell the function how many
(subword)tokens to generate.

In [26]:
def generate_text_by_subword(
    input_str: str,
    model,
    token_mapping: TokenMapping = token_mapping,
    tokenizer = my_tokenizer,
    num_tokens: int = 100,
    temperature: float = 1.0,
    topk: int | None = None,
) -> str:
    # Use your chosen subword-tokenizer
    tokenized_text = tokenize_text_from_tokenizer(
        tokenizer=tokenizer,
        text=input_str,
    )
    # Generates token-by-token and creates a list of those tokens
    generated_tokens = []
    for _ in range(num_tokens):
        # Uses the input text and generated text (so far) to get next token
        new_token = next_token(
            tokenized_text=(tokenized_text + generated_tokens),
            model=model,
            token_mapping=token_mapping,
            # Temperature & top-k sampling used in determining the next token
            temperature=temperature,
            topk=topk,
            device=device,
        )
        generated_tokens.append(new_token)
    # List of all token IDs (input text and generated text)
    output_ids = tokenizer.convert_tokens_to_ids(
        tokenized_text + generated_tokens
    )
    # Returns input string plus the full generated string from list of token IDs
    full_text = tokenizer.decode(output_ids)
    return full_text

## Train Model

At this point, the model has not been trained so the code below will train the
NLP model that will be used to generate new text.

The model will take in the text data (broken by tokens by our subword tokenizer)
and attempt to predict the next token. Over time, the model should hopefully
get better in predicting the next token (given the previous text).

To help us visualize how the model is training, at the end of every epoch, we
generate text using the `TEST_PHRASE` with the improving model.

In [27]:
TEST_PHRASE = 'To be or not to be'
# Use more epochs if not CPU device
epochs = 5 if device == 'cpu' else 25

start = start_time()
for epoch in range(epochs):
    # Set model into "training mode"
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.to(device))
        loss = criterion(output.transpose(1, 2), y_batch.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')
    print(f'[{time_since(start)} ({epoch} {epoch / epochs * 100}) {loss:.4f}]')
    print('-'*72)
    output = generate_text_by_subword(
        input_str=TEST_PHRASE,
        model=model,
        token_mapping=token_mapping,
        tokenizer=my_tokenizer,
        num_tokens=30,
        temperature=1.0,
    )
    print(output)

Epoch 1/25, Loss: 6.491715744922035
 0.4s (0 0.0) 6.2052]
------------------------------------------------------------------------
to be or not to be leap fetchght? bringsoe motion than god ou rude a nice meishmentat new must dearviousuil following simple how slain?ink general pit them
Epoch 2/25, Loss: 5.796372150119982
 0.7s (1 4.0) 5.4181]
------------------------------------------------------------------------
to be or not to be? comes all he, that say'no down wink fai to he, we'' that i, fiery'brave that. nurse there but '
Epoch 3/25, Loss: 5.537153582823904
 0.9s (2 8.0) 5.3079]
------------------------------------------------------------------------
to be or not to be you - the! whom friends d i word cords jul my oforn true wagon i strong. followed, snowbal are spread s i sober ty one
Epoch 4/25, Loss: 5.443774323714407
 1.2s (3 12.0) 5.2120]
------------------------------------------------------------------------
to be or not to be ; in! husband take justice but so forget, hold

## Generate Text

Now that the model has been trained, go ahead and observe how it performs!

Try adjusting the different sampling methods using the `temperature` and `topk`
parameters on the same input string to see the differences.

You might also try different phrases as well as how many tokens (`num_tokens`)
to generate and observe how it does.

------------

Consider how this model differs from the results from the text generation using
the character-based tokenization.

In [28]:
output = generate_text_by_subword(
        input_str='To be or not to be',
        model=model,
        token_mapping=token_mapping,
        tokenizer=my_tokenizer,
        num_tokens=30,
        temperature=1.5,
        topk=100,
    )
print(output)

to be or not to be thy three such, husband who. galloew so'tybalace!, find! nurse., thy ; and ; underneath cousin, loving!
